In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt 


dataset = ['EFIFI','Nair'][1]
basedir = ['../../bf_data/EFIGI_catalog/','../../bf_data/Nair_and_Abraham_2010/'][1]
fitsdir = basedir + ['fits_temp_Jan_19/','fits_temp_Dec_28/', 'fits_temp_Feb_3/', 'final/'][3]

out_dir = basedir+'out_final/'
fns_r = glob(fitsdir+"*/*r.fits")
fns_r.sort()

# Generate mask + Initial StatMorph

In [2]:
from astrobf.utils import gen_mask
from astrobf import measure_morph

In [ ]:
gen_mask.run_mask(fns_r, out_dir, npix_min=40)

/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

0-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log1

1-th chunk done


/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))
/home/hoseung/Work/tonemap/astroBF/astrobf/utils/gen_mask.py:92: RuntimeWarning: divide by zero encountered in log10
  ax.imshow(np.log10(img))


So far, So good.

In [ ]:
# Run StatMorph for the first time
# Just in case, only for this first run, I pickle all 'morph' instances. a few 10s of GB in total.
measure_morph.run_stat_morph_init(fns[:10], out_dir, eps=1e-6)

# Visual inspection - Filter out bad samples
Keep good samples.
Leave good galaxies (under `.../stat_png/good/*.png`) or list bad galaxies (in trash bin).

In [7]:
partially_filtered = "../../bf_data/Nair_and_Abraham_2010/Intermediate_files/final_150000_done.txt"
partial_bad = "../../bf_data/Nair_and_Abraham_2010/Intermediate_files/final_bad_gal_list.txt"

In [35]:
def gids_from_pnglist(plist):
    """
    Parameters
    ----------
        plist : list of png file names
    """
    gids=[]
    for ll in plist:
        gids.append(ll.split('_')[1])
        
    return gids

In [42]:
with open(partial_bad, 'r') as fbad:
    bad = gids_from_pnglist(fbad.readlines())
    
with open(partially_filtered, 'r') as ff:
    temp = gids_from_pnglist(ff.readlines())

good = []

good_gids=[]
for gg in temp:
    if gg not in bad:
        good.append(gg)

In [41]:
len(good)

9537

# Merge files to minimize I/O

In [50]:
from glob import glob
data_dir = '../../bf_data/Nair_and_Abraham_2010/'

fits_fns = glob(data_dir+'fits_final/*/*-r.fits')
weights_fns = glob(data_dir+'fits_final/*/*-r.weight.fits')
mask_fns = glob(data_dir+'final_all_masks/*_mask.pickle')

assert len(mask_fns) == len(fits_fns) == len(weights_fns)

In [65]:
# Make sure three data sets are in the same order
mask_fns.sort()
fits_fns.sort()
weights_fns.sort()

for fn_fits, fn_weight, fn_mask in zip(fits_fns, weights_fns, mask_fns):
    img_name1 = fn_fits.split("/J")[-2] 
    img_name2 = fn_weight.split("/J")[-2] 
    img_name3 = fn_mask.split("/J")[-1].split('_')[0] # mask names are slightly different
    assert img_name1 == img_name2 == img_name3


In [85]:
# merge three dataset into one. 
# make cutout to save storage space
from astropy.io import fits
import pickle
from astrobf.utils import mask_utils

all_good=[]
for fn_fits, fn_weight, fn_mask in zip(fits_fns, weigths_fns, mask_fns):
    img_name = "J"+fn_fits.split("/J")[-2] 
    if img_name in good:
        img = fits.open(fn_fits)[0].data
        weight = fits.open(fn_weight)[0].data
        mask = pickle.load(open(fn_mask, 'rb'))
        slices = mask_utils._get_stamp_range(mask)
        data = np.stack((img[slices], mask[slices], weight[slices]))
        all_good.append({'data':data, 'img_name':img_name, 'slices':slices})

pickle.dump(all_good, open("all_gals.pickle", "wb"))

## Below under construction!!!

### Following part will be updated if I have to go through this step again.

In [ ]:
# Catalog of StatMorph properties of the good sample.
from astrobf.utils.measure_morph import statMorph_fields
fields = statMorph_fields[:6]
ngal = len(good_gids)
result_arr = np.zeros(ngal, 
                      dtype=[('id','<U24'),('ttype',int),('flag',bool),('flag_sersic',bool)]
                           +[(ff,float) for ff in fields])
i=0
for alm in all_morphs:
    mps = pickle.load(open(alm,'rb'))
    for morph in mps:
        if morph._gid in good_gids:
            result_arr[i]['id'] = morph._gid
            for ff in fields:
                result_arr[i][ff] = getattr(morph, ff)
            i+=1
        else:
            pass

# t-types from catalog

Nair catalog has very specific and inconvenient format. You need a dedicated function for this.

In [168]:
cat = utils.misc.load_Nair('../../bf_data/Nair_and_Abraham_2010/catalog/table2.dat')

inds = cat['ID'].searchsorted(result_arr['id'])
print("Is every element matched?: ", np.all(cat[inds]['ID'] == result_arr['id']))
result_arr['ttype'] = cat[inds]['TT']

# Save the complete catalog
pickle.dump(result_arr, open(out_dir+'result_arr.pcikle', 'wb'))

AttributeError: module 'astrobf.utils' has no attribute 'misc'

# Initialize BO loop -  Measure morphology features

In [ ]:
import torch
import math

# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float

### Initial guesses 


# Run Bayesian Optimization loop